In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# Definindo o tamanho do chunk para x linhas
chunksize = 5
reader = pd.read_csv('Quali_20240601.csv', chunksize=chunksize)

# Pegando o primeiro chunk
chunk = next(reader)

# Exibindo o chunk
print(chunk)

  Legal Fund Name Including Umbrella Fund Code Citi Code          ISIN  \
0                               C 11     00F9A      009A  AT0000721402   
1                               C 11     00F9A      009A  AT0000721402   
2                           R-VIP 50     00F9G      009G  AT0000A0FA24   
3                           R-VIP 50     00F9G      009G  AT0000A0FA24   
4                          R-VIP 100     00F9I      009I  AT0000A0F9X2   

      WKN  Apir Code  SEDOL  As Of Date  ESG Score  ESG Rating  ...  \
0  A0J4QW        NaN    NaN         NaN        NaN         NaN  ...   
1  A0J4QW        NaN    NaN         NaN        NaN         NaN  ...   
2  A0YBNA        NaN    NaN         NaN        NaN         NaN  ...   
3  A0YBNA        NaN    NaN         NaN        NaN         NaN  ...   
4  A0YBM7        NaN    NaN         NaN        NaN         NaN  ...   

   BIC Of Transfer Agent  Domicile Of Transfer Agent  Marketmaker Name  \
0                    NaN                         NaN  

In [7]:
with open("colunas_especificas.txt", 'r') as fin:
    cols = fin.readlines()
    cols_especificas = []
    for c in cols:
        cols_especificas.append(c.replace("\n",""))


cols_especificas

['Legal Fund Name Including Umbrella', 'Fund Code', 'Citi Code', 'ISIN', 'WKN', '20000_Financial_Instrument_Identifying_Data', 'EPT Portfolio Name', 'EPT Portfolio or Share Class Currency', 'EPT PRIIPs KID Publication Date', 'EPT Reference Language', 'EPT Valuation Frequency', 'EPT Summary Risk Indicator', 'EPT Is SRI Adjusted', 'EPT Market Risk Measure', 'EPT Credit Risk Measure', 'EPT Recommended Holding Period', 'EPT Number Of Observed Returns', 'EPT Reference Invested Amount', 'EPT Ongoing Costs Management Fees and Other Administrative or Operating Costs', 'EPT Portfolio Transaction Costs', 'EPT Target Market Retail Investor Type', 'EPT Investment Objective', 'EPT Risk Narrative', 'EPT Other Risk Narrative', 'EPT Investment Option', 'Portfolio Turnover Ratio', 'Portfolio Turnover Ratio Date', 'FE Group Name', 'Unit Status', 'Company Contact Address', 'Company Contact E-Mail', 'Company Contact Post Code', 'Company Contact Telephone', 'Company Contact Web Address', 'FE Fund Name', 'S

In [ ]:
# Função para comparar mudanças em chunks

# Função para comparar mudanças em chunks
def compare_chunks(file1, file2, identifiers, columns_to_compare):
    changes = []

    # Ler os arquivos CSV em chunks
    reader1 = pd.read_csv(file1, chunksize=chunksize)
    reader2 = pd.read_csv(file2, chunksize=chunksize)
    #df = pd.concat(reader1, reader2,ignore_index=True)
    # Iterar sobre os chunks de ambos os arquivos
    for chunk1, chunk2 in zip(reader1, reader2):
        # Converta tipos para garantir que são consistentes
        for identifier in identifiers:
            chunk1[identifier] = chunk1[identifier].astype(str)
            chunk2[identifier] = chunk2[identifier].astype(str)

        # Juntar os DataFrames com base nos identificadores
        merged_df = pd.merge(chunk1, chunk2, on=identifiers, suffixes=('_old', '_new'))

        # Analisar mudanças nas colunas específicas
        for column in columns_to_compare:
            old_column = f"{column}_old"
            new_column = f"{column}_new"

            # Verifique se as colunas existem
            if old_column not in merged_df.columns or new_column not in merged_df.columns:
                continue

            # Identifica onde há mudanças
            mask = merged_df[old_column] != merged_df[new_column]
            if mask.any():
                # Adiciona as mudanças ao resultado
                changes.append(merged_df.loc[mask, [*identifiers, old_column, new_column]])

    # Retorna todas as mudanças em um DataFrame
    return pd.concat(changes, ignore_index=True) if changes else pd.DataFrame()

# Defina os arquivos CSV e as colunas
file1 = 'Quali_20240601.csv'
file2 = 'Quali_20240602.csv'
identifiers = ['ISIN', 'Citi Code', 'EPT Reference Language']  # Certifique-se de que os nomes estejam corretos
columns_to_compare = cols_especificas  # As colunas que você deseja comparar

# Chamar a função para comparar os chunks
changes_df = compare_chunks(file1, file2, identifiers, columns_to_compare)

# Exibir mudanças
if not changes_df.empty:
    print("Mudanças encontradas:")
    print(changes_df)
else:
    print("Nenhuma mudança encontrada.")

# Salvar mudanças em um novo CSV, se necessário
# changes_df.to_csv('mudancas.csv', index=False)



# Defina os arquivos CSV e as colunas
file1 = 'Quali_20240601.csv'
file2 = 'Quali_20240602.csv'
identifiers = ['ISIN', 'Citi Code', 'EPT Reference Language']
columns_to_compare = cols_especificas  

# Chamar a função para comparar os chunks
changes_df = compare_chunks(file1, file2, identifiers, columns_to_compare)

# Exibir mudanças
if not changes_df.empty:
    print("Mudanças encontradas:")
    print(changes_df)
else:
    print("Nenhuma mudança encontrada.")

# Salvar mudanças em um novo CSV, se necessário
changes_df.to_csv('mudancas.csv', index=False)
